In [17]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import re #regex pattern matching
import base64 #need for images

from animal_shelter import AnimalShelter #import CRUD Class

###########################
# Data Manipulation / Model
###########################

#authentication
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(password, username)

#create the default dataframe using readAll
df = pd.DataFrame.from_records(shelter.readAll({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#add customers branding
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #create an anchor for the image/logo
    #open the link in a new tab by setting a blank target
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard: Zachary Wright Project Two'))),
    html.Hr(),
    #create radio buttons filter
     html.Div(
        
        #filtering options
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allows row to be selected
        selected_rows=[],
        filter_action="native", #allows a filter
        sort_action="native", #allows sorting
        page_action="native", #enables pagination
        page_current=0, #sets a start page
        page_size=10, #sets rows per page to 10
        
        
    ),
    html.Br(),
    html.Hr(),
    #this sets up the dashboard
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div([
                #dropdown menu for pie chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},       
                            {'label': 'Outcome Type', 'value': 'outcome_type'}
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"10%"}
                ),
                #pie chart
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    
                ),
                #geolocation map
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    )
                ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#filter the data supplied to the dataframe based on which radio button the user selected   
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns'),
               Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #logic to filter interactive data table with MongoDB queries
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.readAll({}))
    elif filter_type == 'WATER':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'MOUNTAIN OR WILDERNESS':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                          "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20},
            "age_upon_outcome_in_weeks":{"$lte": 300},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    #prep data & columns for retrieval from data table
    data=df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    selected_rows = [0] #reset the selected row to first row when filter setting changes
    
    return (data, columns, selected_rows)



#highlight a row on the table when a user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selectedRows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selectedRows]


#populates the pie chart based on dropdown selection.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue)
        )    
    ]


#populates the geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, selectedRows):   
    
    #get selected row or default row
    if not selectedRows:
        selected_row = 0
    else:   
        selected_row = selectedRows[0]
        
    #get data from datatable                           
    dff = pd.DataFrame.from_dict(viewData)   
    
    #get coordinate position data from the selected row
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    
    #get name from selected row
    name = dff.iloc[selected_row,9] 
    if name == "":
        name = "No Name" #replace an empty strings with "No Name"
        
    #return a map using data from the selected row
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX,posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX,posY], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]                           


app #this runs the app